In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Extract data from MIROC5-iso

This notebook is made to plot and extract data at a specific location and time range from MIROC5-iso simulations. Simulations nudged to ERA5 and JRA-55 are available. For now, tritium in precipitation, precipitation rate and 2m air temperature can be donwloaded. 

In [2]:
import os
import math
import numpy as np
import pandas as pd
import xarray as xr
import scipy as sp
import solara
import datetime
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import cartopy.crs as ccrs
#from cartopy.util import add_cyclic_point
#import cartopy.feature as feature
%matplotlib inline

In [3]:
# For the size of the figures
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 300
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')

In [4]:
# some shortcuts to read the miroc5-iso netcdf files
#work = "/Users/acauquoin/Documents/Assistant_Professor_IIS/Tritium/MIROC5-iso_outputs/"
work ="data_tritium_bomb_miroc/"
exp_tritium = "bomb_tritium_"
exp_tritium_era5  = "bomb_tritium_era5"
exp_tritium_jra55 = "bomb_tritium_jra55"

In [5]:
# read netcdf files
#
ifile_prcpTU_era5  = work + "/" + exp_tritium_era5  + "_miroc5_monmean.prcpTU.nc"
ifile_prcp_era5    = work + "/" + exp_tritium_era5  + "_miroc5_monmean.prcp.nc"
ifile_temp2_era5   = work + "/" + exp_tritium_era5  + "_miroc5_monmean.temp2.nc"
ifile_prcpTU_jra55 = work + "/" + exp_tritium_jra55  + "_miroc5_monmean.prcpTU.nc"
ifile_prcp_jra55   = work + "/" + exp_tritium_jra55  + "_miroc5_monmean.prcp.nc"
ifile_temp2_jra55  = work + "/" + exp_tritium_jra55  + "_miroc5_monmean.temp2.nc"
#
ds_prcpTU_era5  = xr.open_dataset(ifile_prcpTU_era5)
ds_prcp_era5    = xr.open_dataset(ifile_prcp_era5)
ds_temp2_era5   = xr.open_dataset(ifile_temp2_era5)
ds_prcpTU_jra55 = xr.open_dataset(ifile_prcpTU_jra55)
ds_prcp_jra55   = xr.open_dataset(ifile_prcp_jra55)
ds_temp2_jra55  = xr.open_dataset(ifile_temp2_jra55)
#
ds_prcpTU_era5  = ds_prcpTU_era5.isel(bnds=0)
ds_prcp_era5    = ds_prcp_era5.isel(bnds=0)
ds_temp2_era5   = ds_temp2_era5.isel(bnds=0)
ds_prcpTU_jra55 = ds_prcpTU_jra55.isel(bnds=0)
ds_prcp_jra55   = ds_prcp_jra55.isel(bnds=0)
ds_temp2_jra55  = ds_temp2_jra55.isel(bnds=0)
#
ds_prcp_era5 = ds_prcp_era5.rename({"pr": "precip"})
ds_temp2_era5 = ds_temp2_era5.rename({"tas": "temp2"})
ds_prcp_jra55 = ds_prcp_jra55.rename({"pr": "precip"})
ds_temp2_jra55 = ds_temp2_jra55.rename({"tas": "temp2"})
#
ds_prcpTU_era5.TU_precip.attrs['units'] = "TU"
ds_prcpTU_era5.TU_precip.attrs['standard_name'] = "tritium in precipitation"
ds_prcp_era5.precip.attrs['units'] = "mm/month"
ds_prcp_era5.precip.attrs['standard_name'] = "precipitation"
ds_temp2_era5.temp2.attrs['units'] = "°C"
ds_temp2_era5.temp2.attrs['standard_name'] = "2m air temperature"
ds_prcpTU_jra55.TU_precip.attrs['units'] = "TU"
ds_prcpTU_jra55.TU_precip.attrs['standard_name'] = "tritium in precipitation"
ds_prcp_jra55.precip.attrs['units']      = "mm/month"
ds_prcp_jra55.precip.attrs['standard_name'] = "precipitation"
ds_temp2_jra55.temp2.attrs['units']      = "°C"
ds_temp2_jra55.temp2.attrs['standard_name'] = "2m air temperature"
#
variables = ['TU_precip', 'precip', 'temp2']
ds_era5  = xr.merge([ds_prcpTU_era5,  ds_prcp_era5,  ds_temp2_era5])
ds_era5 = ds_era5[variables]
ds_jra55 = xr.merge([ds_prcpTU_jra55, ds_prcp_jra55, ds_temp2_jra55])
ds_jra55 = ds_jra55[variables]

In [6]:
def convert360_180(_ds):
    """
    convert longitude from 0-360 to -180 -- 180 deg
    """
    # check if already 
    attrs = _ds['lon'].attrs
    if _ds['lon'].min() >= 0:
        with xr.set_options(keep_attrs=True): 
            _ds.coords['lon'] = (_ds['lon'] + 180) % 360 - 180
        _ds = _ds.sortby('lon')
    return _ds

In [7]:
ds_era5  = convert360_180(ds_era5)
ds_jra55 = convert360_180(ds_jra55)

In [8]:
#f,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
#
#ds_era5['TU_precip'].isel(time=0).plot.pcolormesh(
#    cmap = plt.get_cmap('RdYlBu_r'),ax=ax,
#    cbar_kwargs={
#        'label':'TU', 
#        'orientation':'horizontal'
#    },
#    transform=ccrs.PlateCarree(), 
#    x='lon',y='lat', levels=20)
#ax.set_title('TU in precipitation')
#ax.coastlines()

In [9]:
def plot_and_save_data_site(lon_station, lat_station, 
                            ndg, var, date_min, date_max, 
                            logscale, save_tu, save_precip, save_temp2):
    
    # nudging choice
    if ndg == 'era5':
        ds = ds_era5
    elif ndg == 'jra55':
        ds = ds_jra55
    save_var = [save_tu, save_precip, save_temp2]
    
    # log scale
    if logscale == True:
        yscale = 'log'
    else:
        yscale = 'linear'
    
    # conversion of some variables
    date_min = date_min.strftime("%Y-%m")
    date_max = date_max.strftime("%Y-%m")
    lon_station = float(lon_station)
    lat_station = float(lat_station)
    
    # selection of date
    ds_sel_date = ds.sel(time=slice(date_min, date_max))
    
    # plotting the data on the nearest grid cell of the chosen location
    fig, ax = plt.subplots(figsize=(8,3), constrained_layout=True)
    ax = ds_sel_date[var].sel(lon=lon_station, lat=lat_station, 
                              method='nearest').plot(x='time', 
                                                     ax=ax, yscale=yscale)
    
    # download button
    cols = []
    for i in range(len(variables)):
        if save_var[i] == True:
            cols.append(variables[i])
    ds_sel_date = ds_sel_date.sel(lon=lon_station, lat=lat_station, method='nearest')
    df = ds_sel_date.to_dataframe()
    df = df[cols]
    df.index = df.index.strftime('%Y-%m')
    w = widgets.VBox([solara.FileDownload.widget(data=df.to_csv(sep=';'), filename="data_miroc5-iso.csv")])
    display(w)
    
    # print message
    print("")
    print("Information of the plot:")
    print("Variable:", var)
    print("Longitude:", lon_station, "; Latitude:", lat_station)
    print("Nudging:", ndg)
    print("Date range:", date_min, "to", date_max)
    #print("Longitude resolution:", 360./len(ds.lon))

In [10]:
# update the tritium time series based on coordinates values
def build_ui():
    
    # create widget for coordinates selection
    lon_station = widgets.Text(value="0", 
                               placeholder='between -180 and 180', description='Longitude:', 
                               disabled=False, layout=widgets.Layout(width='200px'))
    lat_station = widgets.Text(value="0", 
                               placeholder='between -90 and 90', description='Latitude:', 
                               disabled=False, layout=widgets.Layout(width='200px'))
    coord_input = lon_station, lat_station
    
    # create widget to select the simulation nudged to ERA5 or JRA-55
    ndg = widgets.Dropdown(options=[('ERA5', 'era5'), ('JRA-55', 'jra55')], value='era5', 
                         description='Nudging:', layout=widgets.Layout(width='250px'))
    
    # create widget to select the variable
    selvar = widgets.Dropdown(options=[('TU in precipitation', 'TU_precip'), 
                                       ('2m air temperature', 'temp2'), 
                                       ('precipitation', 'precip')], 
                              value='TU_precip', description='Variable:',
                              layout=widgets.Layout(width='250px'))
    
    # create a widget to set the y-scale on log or linear scale
    check_logscale = widgets.Checkbox(value=False, description="log scale", indent=False, layout=widgets.Layout(margin='0 0 0 2.5em'))
    
    # create widget to select the variables to save in the csv file
    data = ["TU in precipitation", "Precipitation", "2m air temperature"]
    checkboxes = [widgets.Checkbox(value=False, description=label, indent=False, layout=widgets.Layout(width='150px', margin='0 0 0 0')) for label in data]
    
    # create widget for the date range
    date_min = widgets.DatePicker(description='Date min:', disabled=False, value = datetime.date(1952,1,1))
    date_max = widgets.DatePicker(description='Date max:', disabled=False, value = datetime.date(2018,12,1))
    date_input = date_min, date_max
    
    # Arrange the control inputs:
    controls = widgets.HBox([
        widgets.VBox(coord_input),
        widgets.VBox(date_input),
        widgets.VBox([ndg, selvar]),
        check_logscale
    ])
    
    # link the cdo extraction command with the text input
    plot = widgets.interactive_output(plot_and_save_data_site, 
                                      {'lon_station': lon_station, 'lat_station': lat_station,
                                       'ndg': ndg, 'var': selvar,
                                       'date_min': date_min, 'date_max': date_max,
                                       'logscale': check_logscale,
                                       'save_tu': checkboxes[0], 
                                       'save_precip': checkboxes[1], 
                                       'save_temp2': checkboxes[2]})
    
    # Build the UI panel, the controls at the top, the cdo output at the bottom
    label = widgets.HTML('<b>Select the variables to download:</b>', layout=widgets.Layout(margin='0 1em 0 0'))
    comment = widgets.HTML('Note: the day in the date pickers is not considered.', layout=widgets.Layout(margin='0 1em 0 0'))
    ui = widgets.VBox([widgets.HBox([label, widgets.HBox(children=checkboxes)]), plot, controls, comment])
    
    return ui

In [11]:
build_ui()

In [12]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p numpy,pandas,xarray,solara,ipywidgets,matplotlib,watermark

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.11.2
IPython version      : 8.12.0

numpy     : 1.24.2
pandas    : 1.5.3
xarray    : 2023.3.0
solara    : 1.19.0
ipywidgets: 8.0.6
matplotlib: 3.6.3
watermark : 2.4.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 22.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 10
Architecture: 64bit

 
Last updated: Wed Aug 02 2023 12:17:09JST

